<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔢 Block 1: ติดตั้งไลบรารี (รันครั้งเดียว)
!pip install gspread oauth2client notion-client python-dotenv

In [ ]:
# 🔢 Block 2: Mount Google Drive และโหลด .env
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

# 🔐 path ไปยัง .env
env_path = '/content/drive/MyDrive/Automation_Keys/Notion_Expense/.env'
load_dotenv(dotenv_path=env_path)

In [ ]:
# 🔢 Block 3: Import ไลบรารี + โหลด config จาก .env
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from notion_client import Client

# ✅ โหลดตัวแปรจาก .env
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")
SHEET_ID = os.getenv("SHEET_ID")

# ✅ คำค้นแบบ dynamic สำหรับหมวดหมู่
dynamic_keywords = {
    "อาหาร": ["ข้าว", "แกง", "ก๋วยเตี๋ยว", "กาแฟ", "ชา", "ไข่", "นม", "ขนม", "โจ๊ก"],
    "ของใช้": ["สบู่", "แชมพู", "ผ้า", "ทิชชู่", "ยาสีฟัน", "ครีม"],
    "รถ": ["น้ำมัน", "รถเมล์", "จอดรถ", "เติมลม", "ค่าทางด่วน"],
    "สุขภาพ": ["ยา", "คลินิก", "หมอ", "ฟิตเนส"],
    "บริจาค": ["ทำบุญ", "บริจาค", "ถวาย", "ตักบาตร"],
    "เงินสด": ["ถอน", "เบิก", "รับเงิน", "โอน"],
}

# ✅ Map หมวด → Database ID ที่จะใช้ใน Notion
category_to_db_id = {
    "อาหาร": os.getenv("NOTION_DB_ID_FOOD"),
    "ของใช้": os.getenv("NOTION_DB_ID_SHOPPING"),
    "รถ": os.getenv("NOTION_DB_ID_TRANSPORT"),
    "สุขภาพ": os.getenv("NOTION_DB_ID_HEALTH"),
    "บริจาค": os.getenv("NOTION_DB_ID_DONATION"),
    "เงินสด": os.getenv("NOTION_DB_ID_CASH"),
}

In [ ]:
# 🔢 Block 4: เชื่อม Google Sheets ด้วย gspread
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# 🔐 path ไปยังไฟล์ json ที่อยู่ในโฟลเดอร์เดียวกับ .env
service_account_path = os.path.join(os.path.dirname(env_path), "service_account.json")

# 🔑 เชื่อมต่อ
creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_path, scope)
client = gspread.authorize(creds)

# 📄 เปิด sheet หลัก (sheet1)
sheet = client.open_by_key(SHEET_ID).sheet1
print("📄 Connected to Sheet:", sheet.title)

In [ ]:
# 🔢 Block 5: ฟังก์ชันพื้นฐาน

# 🧾 แยกข้อความออกเป็น หมวด + จำนวนเงิน
def parse_message(text):
    try:
        parts = text.strip().split()
        if len(parts) == 2:
            return parts[0], float(parts[1])
        elif len(parts) == 1:
            return parts[0], 0.0
        else:
            return None, None
    except:
        return None, None

# 🧠 เดาหมวดจาก dynamic_keywords
def find_category(text):
    for category, keywords in dynamic_keywords.items():
        if any(word in text for word in keywords):
            return category
    return None

# 📝 log ข้อความลง Google Sheet
def log_message(user_id, message):
    now = datetime.datetime.now().isoformat()
    sheet.append_row([now, user_id, message, "NO"])

# 🔁 log ข้อความไม่รู้จักลง sheet "Unknown_Categories"
def log_unknown_category(message, parsed_category, amount):
    try:
        unknown_sheet = client.open_by_key(SHEET_ID).worksheet("Unknown_Categories")
    except:
        print("❌ ไม่พบ Sheet 'Unknown_Categories'")
        return

    now = datetime.datetime.now().isoformat()
    unknown_sheet.append_row([now, message, parsed_category or "", amount or ""])
    print(f"📥 Logged unknown: '{message}'")

# 💬 log การสนทนา user/bot
def log_conversation(user_id, role, text):
    try:
        conv_sheet = client.open_by_key(SHEET_ID).worksheet("ConversationLog")
    except:
        print("❌ ไม่พบ Sheet 'ConversationLog'")
        return

    now = datetime.datetime.now().isoformat()
    conv_sheet.append_row([now, user_id, role, text])
    print(f"💬 {role.upper()} | {text}")

In [ ]:
# 🔢 Block 6: เชื่อมต่อ Notion
notion = Client(auth=NOTION_TOKEN)

# 🧾 Push ธรรมดา (มีชื่อหมวดแน่ชัด)
def push_to_notion(category, amount):
    notion.pages.create(
        parent={"database_id": NOTION_DB_ID},
        properties={
            "Categories": {"select": {"name": category}},
            "Amount": {"number": amount}
        }
    )

# 🧠 Push แบบเดาหมวดจากข้อความ
def push_to_notion_dynamic(message):
    category, amount = parse_message(message)
    if not category or amount is None:
        print(f"❌ รูปแบบข้อความไม่ถูกต้อง: '{message}'")
        return

    guessed_category = find_category(category)
    if not guessed_category:
        print(f"⚠️ ไม่พบหมวดที่เหมาะกับ: '{category}'")
        return

    db_id = category_to_db_id.get(guessed_category)
    if not db_id:
        print(f"❌ ไม่พบ Database ID สำหรับ '{guessed_category}'")
        return

    notion.pages.create(
        parent={"database_id": db_id},
        properties={
            "Name": {"title": [{"text": {"content": message}}]},
            "Amount": {"number": amount}
        }
    )
    print(f"✅ ส่งข้อมูลสำเร็จ: '{message}' → หมวด '{guessed_category}'")